In [1]:
import sys
import os
import numpy as np

from codingTools import *

In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
#originalFile="../images/03508649.1_512_512_2_0_12_0_0_0.raw" #int16
#originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
#originalFile="../images/03508649.1_128_128_1_0_8_0_0_0.raw"
#ImageRaw = load_image_raw(originalFile, 1, 128, 128, "uint8")
originalFile = "../images/03508649.1_512_512_2_0_12_0_0_0d.raw"
ImageRaw = load_image_raw(originalFile, 1, 512, 512, ">i2") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [ ]:
########STATISTICS##########
statistics = statistics(original_image)
entropy = statistics.compute_entropy()
print("order zero entropy of the image:", entropy)

print('Max value:', statistics.max_value)
print('Min value:', statistics.min_value)
print('Dynamic range:', statistics.dynamic_range)




In [ ]:
## input bitstream ##
inputbit = binary_file_input('bitpacket.txt')
inputbit.open_binay_file_input()

inputbit.read_value(10)


In [ ]:
## output bitstream ##
outputbit = binary_file_output('test.txt')

value = 999999999
nBits = utils.needed_bits(value)
print('num of bits:', nBits)

outputbit.open_binay_file_output()
outputbit.write_value(value, nBits)
outputbit.close_binary_file_output()

print('size file:',outputbit.size_of_file())

In [ ]:
lista_tridimensional = [
    [
        [7, 12, 8, 10, 6, 4, 34, 68],
        [21, 33, 5, 1, 6, 9, 77, 88],
        [15, 10, 9, 4, 65, 98, 45, 16],
        [11, 19, 27, 29, 44, 37, 66, 52],
        [122, 146, 173, 200, 206, 201, 188, 197],
        [101, 108, 100, 164, 222, 178, 210, 225],
        [50, 30, 20, 5, 199, 240, 202, 190],
        [15, 25, 2, 10, 100, 45, 21, 2],
    ]
]

# Convertir a NumPy array
imagen_prueba = np.array(lista_tridimensional)

# Imprimir el array de NumPy
print(imagen_prueba)

imagen_vacia=np.zeros((1,8,8))
#print(imagen_vacia)

In [ ]:
wav = Wavelet(imagen_prueba,3)
wav_forw = wav.forward(imagen_vacia)
print('forward \n',wav_forw)
print('\n')
wav_inv = wav.inverse(wav_forw,imagen_vacia)
print('inverse \n',wav_inv)

In [ ]:
image_data_empty = ImageRaw.get_image_structure_empty()

print('original imagen \n',original_image)

wav = Wavelet(original_image,3)

matriz_wavelet = wav.forward(image_data_empty)

print('matriz forward Wavelet\n',matriz_wavelet)

matriz_recuperada = wav.inverse(matriz_wavelet,image_data_empty)

print('matriz inverse wavelet \n',matriz_recuperada)



In [ ]:
image_data_empty = ImageRaw.get_image_structure_empty()
pred1 = Prediction(original_image)
imagen_prima1 = pred1.north_and_east(image_data_empty)
residu1 = original_image-imagen_prima1
print(residu1)

q = Quantizer(1,10)
image_quantized = ImageRaw.get_image_structure_empty()
q.quantize(residu1,image_quantized)# image quantized = r prima !!hem de treballar amb aquesta

image_DEquantized = ImageRaw.get_image_structure_empty()
q.dequantize(image_quantized,image_DEquantized) # image desquantized = r barret

aux_image = image_DEquantized + imagen_prima1  # utilitzar rprima en el predictor 

print(aux_image)


m = Metrics(original_image,aux_image)
m.compute()



In [ ]:

statistics1 = statistics(original_image)
entropy1 = statistics1.compute_entropy()
print("Entropia imagen original:", entropy1)

statistics4 = statistics(residu1)
entropy4 = statistics4.compute_entropy()
print("Entropia predictor north and east (residu):", entropy4)

statistics5 = statistics(image_quantized)
entropy5 = statistics5.compute_entropy()
print("Entropia predictor north and east (quantizado):", entropy5)


In [3]:
image_data_empty = ImageRaw.get_image_structure_empty()
q=Quantizer(1,10)
'''
pred1 = Prediction(original_image)
imagen_prima1 = pred1.north_and_east(image_data_empty)
residu1 = original_image-imagen_prima1
    
q=Quantizer(1,10)
img_q=ImageRaw.get_image_structure_empty()
q.quantize(residu1, img_q)
        
img_desq=ImageRaw.get_image_structure_empty()
q.dequantize(img_q, img_desq)
'''
        
dimension = image_data_empty.shape[0]
filas = image_data_empty.shape[1]
columnas = image_data_empty.shape[2]
#result = np.copy(empty_image)
        
p=ImageRaw.get_image_structure_empty()
x_barret=ImageRaw.get_image_structure_empty()
#r = ImageRaw.get_image_structure_empty()
r_prima = 0#ImageRaw.get_image_structure_empty()
r_2prima = 0#ImageRaw.get_image_structure_empty()
        
for dim in range(dimension):
    for i in range(filas):
        for j in range(columnas):     
            if j==0:
                p[dim][i][j]=0
                #r[dim][i][j]=
                r = original_image[dim][i][j] - p[dim][i][j] #codigo profe
                r_prima= r//10 
                #q.quantize(r,r_prima)
                r_2prima = r_prima*10
                #q.dequantize(r_prima,r_2prima)
                x_barret[dim][i][j] =  p[dim][i][j] + r_2prima #[dim][i][j]  #codigo profe
                
            else:  
                #codigo nuestro
                #p[dim][i][j]=p[dim][i][j-1]+img_desq[dim][i][j]
                
                p[dim][i][j] = original_image[dim][i][j-1] #codigo profe
                #r[dim][i][j]
                r = original_image[dim][i][j] - p[dim][i][j] #codigo profe
                r_prima= r//10
                #q.quantize(r,r_prima)
                r_2prima = r_prima*10
                #q.dequantize(r_prima,r_2prima)
                x_barret[dim][i][j] =  p[dim][i][j] + r_2prima #[dim][i][j]  #codigo profe
        
#x_barret=img_desq+p

In [4]:
image_data_empty = ImageRaw.get_image_structure_empty()
pred1 = Prediction(x_barret)
imagen_prima1 = pred1.north_and_east(image_data_empty)
residu1 = original_image-imagen_prima1
    
q=Quantizer(1,10)
img_q=ImageRaw.get_image_structure_empty()
q.quantize(residu1, img_q)
        
img_desq=ImageRaw.get_image_structure_empty()
q.dequantize(img_q, img_desq)
        
aux_img=img_desq+imagen_prima1
m=Metrics(original_image, aux_img)
m.compute()

bits 12
MSE= 28.512107849121094
PSNR= 57.69478487082531
PAE= 9.0
